# Trabajo en clases: Web Scrapping
Nombre: Rossy Armendariz

En base a la página web all recipes, sustraer al menos 200 recetas de las primeras 10 categorias.

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL principal de categorías
categories_url = "https://www.allrecipes.com/recipes-a-z-6735880"

# Función para obtener categorías y sus URLs
def get_categories(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error al acceder a la página de categorías: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    categories = soup.find_all("a", href=True)

    # Filtrar enlaces válidos de categorías
    category_links = {cat.text.strip(): cat['href'] for cat in categories if "/recipes/" in cat['href']}
    return category_links

# Ejecutar el proceso para obtener las categorías
categories = get_categories(categories_url)
if categories:
    # Seleccionar solo las primeras 10 categorías
    selected_categories = dict(list(categories.items())[:10])

    # Guardar en un archivo CSV
    categories_df = pd.DataFrame(list(selected_categories.items()), columns=["Category", "Category URL"])
    categories_df.to_csv("categories_list.csv", index=False)
    print("Archivo 'categories_list.csv' generado con las primeras 10 categorías.")


Archivo 'categories_list.csv' generado con las primeras 10 categorías.


Obtención de las URLs y IDs de las recetas de las categorías.

In [10]:
import re
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Cargar categorías desde el archivo generado en el paso 1
categories_df = pd.read_csv("categories_list.csv")
categories = dict(zip(categories_df["Category"], categories_df["Category URL"]))

# Función para extraer recetas de una categoría
def get_recipes_by_category(category_name, category_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(category_url, headers=headers)

    # Verificar que la respuesta sea exitosa
    if response.status_code != 200:
        print(f"Error al acceder a la categoría '{category_name}': {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")

    # Buscar todas las tarjetas de recetas
    recipes = soup.find_all("a", href=True)

    # Filtrar solo enlaces válidos de recetas
    recipe_links = []
    for rec in recipes:
        recipe_url = rec['href']

        # Verificar que la URL sea válida para recetas en ambos formatos
        if re.search(r"/recipe/\d+/|/recipe-\d+", recipe_url):
            recipe_links.append(recipe_url)

    return recipe_links

# Procesar todas las categorías para obtener las recetas
recipes_summary = []
recipe_urls = []
for category, category_url in categories.items():
    print(f"Procesando recetas de la categoría: {category}")
    recipe_links = get_recipes_by_category(category, category_url)
    for recipe_url in recipe_links:
        # Extraer ID de la receta desde la URL en ambos formatos
        recipe_id_match = re.search(r"/recipe/(\d+)/|/recipe-(\d+)", recipe_url)
        recipe_id = recipe_id_match.group(1) if recipe_id_match and recipe_id_match.group(1) else recipe_id_match.group(2) if recipe_id_match else "No ID Found"

        # Agregar a resumen de recetas
        recipes_summary.append({
            "Category": category,
            "ID": recipe_id,
            "Recipe URL": recipe_url,
        })

        # Agregar solo la URL a la lista de URLs
        recipe_urls.append({"Recipe URL": recipe_url})

    time.sleep(1)  # Evitar bloqueos

# Guardar los resultados en archivos
# Documento 1: Solo las URLs
recipe_urls_df = pd.DataFrame(recipe_urls)
recipe_urls_df.to_csv("recipe_urls.csv", index=False)
print("Archivo 'recipe_urls.csv' generado con las URLs de las recetas.")

# Documento 2: Categoría, ID y URL
recipes_summary_df = pd.DataFrame(recipes_summary)
recipes_summary_df.to_csv("recipes_summary.csv", index=False)
print("Archivo 'recipes_summary.csv' generado con las categorías, IDs y URLs.")


Procesando recetas de la categoría: Dinners
Procesando recetas de la categoría: 5-Ingredient Dinners
Procesando recetas de la categoría: One-Pot Meals
Procesando recetas de la categoría: Quick & Easy
Procesando recetas de la categoría: 30-Minute Meals
Procesando recetas de la categoría: Family Dinners
Procesando recetas de la categoría: Soups, Stews & Chili
Procesando recetas de la categoría: Comfort Food
Procesando recetas de la categoría: Main Dishes
Procesando recetas de la categoría: Sheet Pan Dinners
Archivo 'recipe_urls.csv' generado con las URLs de las recetas.
Archivo 'recipes_summary.csv' generado con las categorías, IDs y URLs.


Obtención de los siguientes resultados por receta:
- ID
- Nombre de la receta
- Detalle de la receta
- Ingredientes
- Preparación
- URL de la receta

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# Cargar el archivo con las URLs de recetas (incluyendo el ID)
recipe_urls_df = pd.read_csv("recipes_summary.csv")
recipe_data = recipe_urls_df[["ID", "Recipe URL"]].to_dict(orient="records")

# Función para extraer detalles de una receta
def get_recipe_details(recipe_id, recipe_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(recipe_url, headers=headers)

    # Verificar que la respuesta sea exitosa
    if response.status_code != 200:
        print(f"Error al acceder a la receta (ID {recipe_id}): {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, "html.parser")

    # Extraer título de la receta
    title_element = soup.find("h1", class_="article-heading text-headline-400")
    title = title_element.text.strip() if title_element else "No Title Found"

    # Extraer descripción de la receta
    description_element = soup.find("p", class_="article-subheading text-body-100")
    description = description_element.text.strip() if description_element else "No Description Found"

    # Extraer ingredientes
    ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
    ingredients = [ingredient.text.strip() for ingredient in ingredients_section]

    # Extraer pasos de preparación
    steps_section = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
    steps = [step.text.strip() for step in steps_section]

    return {
        "ID": recipe_id,
        "Recipe Name": title,
        "Description": description,
        "Ingredients": "; ".join(ingredients),
        "Preparation Steps": " ".join(steps),  # Combina todos los pasos en una sola cadena
        "Recipe URL": recipe_url,
    }

# Procesar todas las URLs de recetas para obtener detalles
recipe_details = []
for recipe in recipe_data:
    recipe_id = recipe["ID"]
    recipe_url = recipe["Recipe URL"]
    print(f"Procesando receta (ID {recipe_id}): {recipe_url}")
    details = get_recipe_details(recipe_id, recipe_url)
    if details:
        recipe_details.append(details)
    time.sleep(1)  # Evitar bloqueos

# Guardar los detalles en un archivo CSV
recipe_details_df = pd.DataFrame(recipe_details)
# Asegurarse de que el ID sea la primera columna
recipe_details_df = recipe_details_df[["ID", "Recipe Name", "Description", "Ingredients", "Preparation Steps", "Recipe URL"]]
recipe_details_df.to_csv("recipe_details.csv", index=False)
print("Archivo 'recipe_details.csv' generado con los detalles de las recetas, incluyendo el ID de la receta.")


Procesando receta (ID 212498): https://www.allrecipes.com/recipe/212498/easy-chicken-and-broccoli-alfredo/
Procesando receta (ID 83646): https://www.allrecipes.com/recipe/83646/corned-beef-roast/
Procesando receta (ID 158799): https://www.allrecipes.com/recipe/158799/stout-braised-lamb-shanks/
Procesando receta (ID 8509102): https://www.allrecipes.com/recipe/8509102/chicken-al-pastor/
Procesando receta (ID 8508920): https://www.allrecipes.com/recipe/8508920/mississippi-chicken/
Procesando receta (ID 255462): https://www.allrecipes.com/recipe/255462/lasagna-flatbread/
Procesando receta (ID 245210): https://www.allrecipes.com/recipe/245210/prosciutto-wrapped-pork-tenderloin-with-crispy-sage/
Procesando receta (ID 215231): https://www.allrecipes.com/recipe/215231/empanadas-beef-turnovers/
Procesando receta (ID 268494): https://www.allrecipes.com/recipe/268494/basic-air-fryer-hot-dogs/
Procesando receta (ID 11679): https://www.allrecipes.com/recipe/11679/homemade-mac-and-cheese/
Procesando

## Preprocesamiento de los datos

In [13]:
import csv
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk

# Descargar las stopwords de NLTK si no lo has hecho antes
nltk.download('stopwords')

# Configuración
archivo_original = './recipe_details.csv'
archivo_salida = 'recipe_details_procesado.csv'
stop_words = set(stopwords.words('english'))  # Cambia a 'english' si es necesario
stemmer = SnowballStemmer('english')  # Cambia el idioma a 'english' si es necesario

# Función para procesar el texto
def procesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar signos de puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    # Eliminar stopwords y aplicar stemming
    palabras = texto.split()
    palabras_procesadas = [
        stemmer.stem(palabra) for palabra in palabras if palabra not in stop_words
    ]
    return ' '.join(palabras_procesadas)

# Leer el archivo original y procesar
with open(archivo_original, mode='r', encoding='utf-8') as entrada:
    lector = csv.reader(entrada)
    filas_procesadas = []

    # Procesar la primera fila (nombres de las columnas) sin modificaciones
    encabezados = next(lector)  # Obtener la primera fila
    filas_procesadas.append(encabezados)  # Agregarla directamente al resultado

    # Procesar las demás filas
    for fila in lector:
        fila_procesada = [procesar_texto(celda) for celda in fila]
        filas_procesadas.append(fila_procesada)

# Escribir el archivo procesado
with open(archivo_salida, mode='w', encoding='utf-8', newline='') as salida:
    escritor = csv.writer(salida)
    escritor.writerows(filas_procesadas)

print(f"El archivo procesado ha sido guardado como '{archivo_salida}'.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


El archivo procesado ha sido guardado como 'recipe_details_procesado.csv'.


Instalación de las bibliotecas necesarias para utilizar FAISS para la recuperación.

In [2]:
%pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.2 MB/s eta 0:00:00


Creación de embeddings, índice de FAISS y la función de búsqueda con FAISS.

In [7]:
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Cargar los archivos CSV
preprocessed_df = pd.read_csv("recipe_details_procesado.csv")  # Archivo preprocesado
original_df = pd.read_csv("recipe_details.csv")  # Archivo sin preprocesar
original_df.columns = ["ID", "Recipe Name", "Description", "Ingredients", "Preparation Steps", "Recipe URL"]

# Asegurar que los IDs sean tratados como strings
preprocessed_df["ID"] = preprocessed_df["ID"].astype(str)
original_df["ID"] = original_df["ID"].astype(str)

# Modelo de embeddings
print("Cargando el modelo de embeddings...")
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings a partir de las descripciones procesadas
print("Generando embeddings...")
descripciones = preprocessed_df["Description"].tolist()
embeddings = modelo_embeddings.encode(descripciones, show_progress_bar=True)

# Crear índice FAISS
dimension = embeddings.shape[1]  # Dimensión de los vectores
index = faiss.IndexFlatL2(dimension)  # Índice con métrica de distancia Euclidiana
index.add(embeddings)  # Agregar los embeddings al índice

# Confirmar que los vectores se han agregado correctamente
print(f"Total de vectores en el índice: {index.ntotal}")

# Función para realizar búsquedas con FAISS
def realizar_busqueda_faiss(consulta, num_resultados=5):
    # Generar el embedding de la consulta
    consulta_embedding = modelo_embeddings.encode([consulta])

    # Buscar los vecinos más cercanos
    distancias, indices = index.search(consulta_embedding, num_resultados)

    # Recuperar resultados del archivo preprocesado
    resultados_preprocesados = preprocessed_df.iloc[indices[0]].copy()
    resultados_preprocesados["Distancia"] = distancias[0]

    # Crear una lista para almacenar los resultados originales
    resultados_completos = []

    # Buscar los datos originales usando los IDs
    for _, fila in resultados_preprocesados.iterrows():
        receta_original = original_df[original_df["ID"] == fila["ID"]]
        if not receta_original.empty:
            resultados_completos.append({
                "ID": fila["ID"],
                "Recipe Name": receta_original.iloc[0]["Recipe Name"],
                "Description": receta_original.iloc[0]["Description"],
                "Ingredients": receta_original.iloc[0]["Ingredients"],
                "Preparation Steps": receta_original.iloc[0]["Preparation Steps"],
                "Recipe URL": receta_original.iloc[0]["Recipe URL"],
                "Distancia": fila["Distancia"]
            })

    # Convertir los resultados completos en un DataFrame
    resultados_df = pd.DataFrame(resultados_completos)
    return resultados_df

Cargando el modelo de embeddings...
Generando embeddings...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Total de vectores en el índice: 344


Ejemplo práctico de la consulta mediante la recuperación FAISS:

In [14]:
# Ejemplo práctico de consulta
consulta = "chicken"
print("\nResultados de la consulta:")
resultados = realizar_busqueda_faiss(consulta, num_resultados=5)

# Mostrar los resultados
print(resultados)



Resultados de la consulta:
       ID                              Recipe Name  \
0  220751                    Quick Chicken Piccata   
1  236306                    Lemon-Roasted Chicken   
2  260625  Crumbed Chicken Tenderloins (Air Fried)   
3  260625  Crumbed Chicken Tenderloins (Air Fried)   
4  176132   Slow Cooker Buffalo Chicken Sandwiches   

                                         Description  \
0  Chicken piccata is super quick and easy to mak...   
1  This lemon-roasted chicken is our go-to baked ...   
2  These air fryer chicken tenderloins beat chick...   
3  These air fryer chicken tenderloins beat chick...   
4  This crockpot Buffalo chicken is great for hea...   

                                         Ingredients  \
0  4  skinless, boneless chicken breast halves; c...   
1  1 whole chicken, cut into 8 pieces; 1 onion, c...   
2  1 large egg; ½ cup dry bread crumbs; 2 tablesp...   
3  1 large egg; ½ cup dry bread crumbs; 2 tablesp...   
4  4  skinless, boneless chick